In [1]:
#import dependencies 
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, "html.parser")

In [5]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text
news_title = soup.find("div", class_= "content_title").text
news_p = soup.find("div", class_= "article_teaser_body").text
print(news_title)
print(news_p)

All About the Laser (and Microphone) Atop Mars 2020, NASA's Next Rover
SuperCam is a rock-vaporizing instrument that will help scientists hunt for Mars fossils.


In [6]:
#Visit the url for JPL Featured Space Image
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

In [7]:
html = browser.html
soup = bs(html, "html.parser")

In [8]:
#collect and print featured image url
featured_image_url = []
images = soup.find_all("a", class_= "fancybox")
for image in images:
    image = image['data-fancybox-href']
    featured_image_url.append(image)

featured_image = "https://www.jpl.nasa.gov"+ featured_image_url[0]
print(featured_image)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19968_ip.jpg


In [9]:
#Visit the Mars Weather twitter account
mars_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_url)

In [10]:
response = requests.get(mars_url)
soup = bs(response.text, "html.parser")

In [11]:
#scrape the latest Mars weather tweet from the page
mars_weather = []
tweets = soup.find_all("div", class_= "content")
for tweet in tweets:
    tweet = tweet.find("div", class_= "js-tweet-text-container").text
    mars_weather.append(tweet)

print(mars_weather[0])


Happy 150th Birthday to the National Weather Service. Ready to open Mars offices when you are. Should have the Jezero crater site up and running this time next year. #Mars2020
https://vlab.ncep.noaa.gov/web/nws-heritage …pic.twitter.com/UeAu31btC8



In [14]:
#Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet
mars_facts_url = "https://space-facts.com/mars/"
browser.visit(mars_facts_url)
html = browser.html
fact_table = pd.read_html(mars_facts_url)
mars_df = fact_table[0]
mars_df.columns = ["Description", "Value"]
mars_df.set_index("Description", inplace=True)
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
#convert the data to a HTML table string
mars_html = mars_df.to_html("table.html")

In [16]:
#visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.
mars_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_url)

In [17]:
html = browser.html
soup = bs(html, "html.parser")

In [18]:
#create empty list to store dictionaries of titles & links to images
hemisphere_image_urls = []

#retrieve all elements that contain image information
results = soup.find("div", class_ = "result-list" )
hemispheres = results.find_all("div", class_="item")

#loop through each image
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    #remove Enhanced from image title 
    title = title.replace(" Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})


hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]